In [ ]:
# !pip install datasets evaluate transformers[sentencepiece] -q
# !pip install accelerate -q
!pip install git+https://github.com/csebuetnlp/normalizer -q

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import os
all_data={}
for x in os.listdir("data"):
    print(x.split(".")[0])
    try:
        raw_datasets = load_dataset("json",data_files=f"data/{x}")
        all_data[x.split(".")[0]]=raw_datasets
    except:
        continue

In [ ]:
f=open("category.txt","r").read().split("\n")
dic={}
for x in f:
    xx=x.split(";")
    dic[xx[1].replace('\'',"")]=xx[2].replace('\'',"")

In [ ]:
dic_all=[]
for x in dic:
    dic_all.append(x)
    dic_all.append(dic[x])

In [ ]:
def check(ex):
    if ex['Category'] in dic_all:
        return True
    else:
        return False
all_data2={}
for x in all_data:
    single_data=all_data[x]
    single_data=single_data.filter(check)
    all_data2[x]=single_data


In [ ]:
for x in all_data2:
    print(all_data2[x])

In [ ]:
all_data3={}
def fun(exam):
    exam['Category']=dic[exam['Category']]
    return {'Category': exam['Category'], 'Content': exam['Content']}
for x in all_data2:
    single_data=all_data2[x]
    print(single_data)
    single_data=single_data.map(fun)
    print(single_data)
    all_data3[x]=single_data

In [ ]:
for x in all_data3:
    # print(all_data3[x])
    print(x)
    break

In [ ]:
# def select_features(example):
imp_feat=['Category','Content']    
for x in all_data3:
    tt=(all_data3[x]['train'].features)
    for x2 in tt:
        if x2 not in imp_feat:
            all_data3[x]['train']=all_data3[x]['train'].remove_columns(x2)
    
        
    

In [27]:
for x in all_data3:
    print(set(all_data3[x]['train']['Category']))

{'বিনোদন', 'শিক্ষা', 'আন্তর্জাতিক', 'খেলা', 'স্বাস্থ্য', 'অর্থনীতি', 'তথ্য ও প্রযুক্তি', 'রাজনীতি', 'বাংলাদেশ'}
{'শিক্ষা', 'বিনোদন', 'আন্তর্জাতিক', 'খেলা', 'তথ্য ও প্রযুক্তি', 'অর্থনীতি', 'রাজনীতি', 'স্বাস্থ্য', 'বাংলাদেশ'}
{'বিনোদন', 'শিক্ষা', 'তথ্য ও প্রযুক্তি', 'আন্তর্জাতিক', 'খেলা', 'অর্থনীতি', 'রাজনীতি', 'বাংলাদেশ'}
{'বিনোদন', 'শিক্ষা', 'আন্তর্জাতিক', 'খেলা', 'স্বাস্থ্য', 'অর্থনীতি', 'রাজনীতি', 'তথ্য ও প্রযুক্তি', 'বাংলাদেশ'}
{'বিনোদন', 'শিক্ষা', 'আন্তর্জাতিক', 'খেলা', 'তথ্য ও প্রযুক্তি', 'রাজনীতি', 'অর্থনীতি', 'স্বাস্থ্য', 'বাংলাদেশ'}


In [ ]:
from datasets import concatenate_datasets
temp_li=[]
for x in all_data3:
    temp_li.append(all_data3[x]['train'])
dd=concatenate_datasets(temp_li)



In [30]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
d = {'train':Dataset.from_dict({'Category':dd['Category'],'Content':dd['Content']})}
raw_datasets=DatasetDict(d)

In [31]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Category', 'Content'],
        num_rows: 731945
    })
})

In [32]:
from normalizer import normalize
def normalize_fun(input_text):
    inp=input_text['Content']
#     input_text = data['text'][0]
    normalized_text = normalize(
        inp,
        unicode_norm="NFKC",          # type of unicode normalization (default "NFKC")
        punct_replacement=None,       # an optional string or callable for replacing the punctuations (default `None`, i.e. no replacement)
        url_replacement=None,         # an optional string or callable for replacing the URLS (default `None`, i.e. no replacement)
        emoji_replacement=None,       # an optional string or callable for replacing the emojis (default `None`, i.e. no replacement)
        apply_unicode_norm_last=True  # whether to apply the unicode normalization before or after rule based replacements (default True)
    )
    input_text['Content']=inp
    return input_text

In [33]:
raw_datasets['train']=raw_datasets['train'].map(normalize_fun)

Map:   0%|          | 0/731945 [00:00<?, ? examples/s]

In [34]:
s=set()
for idx,x in enumerate(raw_datasets['train']):
  s.add(raw_datasets['train'][idx]["Category"])

In [35]:
label_dict={}
for id,x in enumerate(s):
  label_dict[x]=id
label_dict


{'বিনোদন': 0,
 'শিক্ষা': 1,
 'আন্তর্জাতিক': 2,
 'খেলা': 3,
 'স্বাস্থ্য': 4,
 'অর্থনীতি': 5,
 'তথ্য ও প্রযুক্তি': 6,
 'রাজনীতি': 7,
 'বাংলাদেশ': 8}

In [36]:
def assign_label(example):
  example['Category']=label_dict[example['Category']]
  return example
raw_datasets['train']=raw_datasets['train'].map(assign_label)

Map:   0%|          | 0/731945 [00:00<?, ? examples/s]

In [37]:
checkpoint = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [38]:
# model.config.max_position_embeddings

In [39]:
def tokenize_function(example):
    return tokenizer(example["Content"], truncation=True,max_length=512)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/731945 [00:00<?, ? examples/s]

In [40]:
tokenized_datasets = tokenized_datasets.remove_columns(['Content'])

In [41]:
tokenized_datasets = tokenized_datasets.rename_column("Category", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [42]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [43]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [44]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 418]),
 'token_type_ids': torch.Size([8, 418]),
 'attention_mask': torch.Size([8, 418])}

In [45]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=9)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(2.2903, grad_fn=<NllLossBackward0>) torch.Size([8, 9])


In [47]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/home/rtx3090/anaconda3/envs/LLMTesting/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [48]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

274482


In [49]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [50]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/274482 [00:00<?, ?it/s]

In [51]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')